In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from src.prepare_data import prepare_model_data



In [2]:
import pandas as pd

# Load raw data
df_raw = pd.read_csv('../data/row/data.csv')

# Prepare model-ready data
df_ready = prepare_model_data(df_raw)

df_ready.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,TransactionStartTime,PricingStrategy,FraudResult,recency,frequency,monetary,transaction_hour,transaction_day,transaction_month,transaction_year
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256.0,ProviderId_6,ProductId_10,airtime,...,2018-11-15 02:18:49+00:00,2.0,0.0,-0.746427,-0.311831,0.170118,2.0,15.0,11.0,2018.0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256.0,ProviderId_4,ProductId_6,financial_services,...,2018-11-15 02:19:08+00:00,2.0,0.0,-0.746427,-0.311831,0.170118,2.0,15.0,11.0,2018.0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256.0,ProviderId_6,ProductId_1,airtime,...,2018-11-15 02:44:21+00:00,2.0,0.0,3.383499,-0.444993,0.165122,2.0,15.0,11.0,2018.0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256.0,ProviderId_1,ProductId_21,utility_bill,...,2018-11-15 03:32:55+00:00,2.0,0.0,-0.491494,-0.404020,0.175567,3.0,15.0,11.0,2018.0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256.0,ProviderId_4,ProductId_6,financial_services,...,2018-11-15 03:34:21+00:00,2.0,0.0,-0.491494,-0.404020,0.175567,3.0,15.0,11.0,2018.0


In [3]:
df_ready.to_csv('../data/processed/model_input.csv', index=False)

print(df_ready.head())

         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX        256.0  ProviderId_6  ProductId_10   
1  CustomerId_4406          UGX        256.0  ProviderId_4   ProductId_6   
2  CustomerId_4683          UGX        256.0  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX        256.0  ProviderId_1  ProductId_21   
4   CustomerId_988          UGX        256.0  ProviderId_4   ProductId_6   

      ProductCategory  ...      TransactionStartTime  PricingStrateg

In [4]:
from src.data_processing.rfm_clustering import (
    cluster_customers_rfm,
    assign_high_risk_label
)

# Extract unique RFM rows
rfm_df = df_ready[['CustomerId', 'recency', 'frequency', 'monetary']].drop_duplicates()

# Cluster customers
rfm_clustered = cluster_customers_rfm(rfm_df)

# Assign proxy target
rfm_labeled = assign_high_risk_label(rfm_clustered)

# Merge back to full dataset
df_final = df_ready.merge(
    rfm_labeled[['CustomerId', 'is_high_risk']],
    on='CustomerId',
    how='left'
)


In [6]:
df_final.to_csv('../data/processed/model_training_data.csv', index=False)
